<a href="https://colab.research.google.com/github/csrajath/CSC_8980_NLP/blob/main/csc_8980_NLP_exam1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Name: **Rajath Chikkatur Srinivasa** \
PantherID: **002552425**

In [17]:
# importing required libraries
import re, random, os, nltk,spacy, os
from collections import Counter, defaultdict
import pandas as pd
import numpy as np
import seaborn as sns
from nltk import bigrams, trigrams
# all sklearn imports
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import *
# importing models
#importing classifier types
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Perceptron
from sklearn.pipeline import make_pipeline

In [4]:
# unzipping dataset
# !unzip /content/exam1_dataset.zip
# !tar -xvf /content/exam1_dataset.zip
!unzip /content/drive/MyDrive/Projects/NLP_Exam/exam1_dataset.zip

**Question 1) (20 points) Write a generic function that takes: Classification algorithm name, vectorization method name, training set with labels as parameters (total of 3 parameters should be passed). The function should take the classification algorithm name, the vectorization method’s name, and the training set and train the desired model. Use the default training parameters for the models we have seen in class. This function should return the trained model**

In [ ]:
def generic_fun(algo_name, vec_method, train_set):
  pass

**Question 2) (30 points) Using the function from question 1 to build the following models:
a) Model a: Naive Bayes, Vectorizer: TFIDF and Bag of Words, Training set should be 75% of the provided dataset. Leaving the remaining 25% for testing.
b) Model b: RandomForest, Vectorizer: TFIDF and Bag of Words, Training set should be 70% of the provided dataset. Leaving the remaining 30% for testing.
c) Model c: Support Vector Machines (SVC in sklearn), Vectorizer: TFIDF and Bag of Words, Training set should be 60% of the provided dataset. Leaving the remaining 40% for testing. NOTE: Set the random seed to: 12345. This needs to be consistently set to train the model AND split the data in test and train. If this is not done correctly, you will lose points as your answers will not be comparable with the grading key**

**Question 3) (30 points) Using the models from Question 2, evaluate each model with its respective training set (for model a, that set is 25% of the data, for model b, that set is 30% of the data, and for model c that set is 40% of the data. Be careful to not mix up the evaluation sets. With the predictions on the test set and show the following metrics: Accuracy, Precision, Recall, and Macro F1-score. With this in mind, please write and answer these questions in your notebook:
a) What model performs the best and why? (which metrics do you base this on, and why do you think it performs better than others).
b) Why is it important not to mix up the testing sets between different models? Think about this one.
c) Display in a single sorted dataframe (model name, training %, test %, accuracy, precision, recall, F1-score) all performance metrics, sorted by accuracy in descending
manner**

**Question 4) (15 points) Using the documents in the folder named UNLABELED, please use your best performing trained model from question 3 to predict their labels. Please do this individually for each document. Print to the screen the following items: Document Name, Predicted Label and using a text cell, write your own opinion if the label is correct and why - note you have to read the document to make your own opinion**

**Question 5) (20 points) Build a function that takes the set of documents as input and returns a cosine similarity matrix for those documents. Feed all documents in the TRAINING folder to this matrix. Instead of printing the returned cosine similarity matrix, create a heatmap plot from the returned matrix. Make sure your plot is nicely scaled, properly labeled, and uses a nice color range to show the similarity**

In [18]:
# creating set of documents i.e. list of all documents present in TRAINING folder
folder_path = '/content/exam1_dataset/TRAINING/'
file_content_corpus = []
# print(distance())
sub_folders = []
for i in os.walk(folder_path):
  sub_folders.append(i[0])
for folder in sub_folders:
  files = os.walk(folder).__next__()[-1]   
  for f in files:
    if not f.startswith('.'):                                                      
      file_content_corpus.append(open(os.path.join(folder, f),'r').read()) 

In [4]:
#driver code / function
def distance(docset):
  vec = TfidfVectorizer()
  X = vec.fit_transform(docset)
  cos_s = cosine_similarity(X, X, dense_output=False)
  return cos_s

In [5]:
# function output stored in conf_mat variable
cos_mat = distance(file_content_corpus)

In [ ]:
cos_mat.todense()

In [13]:
# #creating heatmap
# plt.figure(figsize=(6,7))
sns.heatmap(cos_mat, square=True, annot=True, fmt='d', cbar=False)
plt.xlabel('true label')
plt.ylabel('predicted label');

**Question 6) (15 points) Write a function that takes a cosine similarity matrix as input and returns a list with the top n document paris and their similarity. Note that you should only keep the document pairs that are unique and remove the comparisons of the document to itself. Print the top 50 similar document pairs. Compare the assigned class for each document and answer: Do all similar documents belong to the same class? Why or why not?**

In [1]:
def top_n_similar_works(cos_matrix, n):
  topNSimilar = []
  np.fill_diagonal(cos_matrix, 0)
  index_1d = cos_matrix.flatten().argsort()[-(2*n):]
  x_idx, y_idx = np.unravel_index(index_1d, cos_matrix.shape)
  for i in range(0,len(x_idx),2):
    x = x_idx[i]
    y = y_idx[i]
    if(x != y):
      topNSimilar.append((x,y,cos_matrix[x][y]))
  return topNSimilar[::-1]

In [ ]:
print('top 50 similar works\n')
for i in top_n_similar_works(cos_s,50):
  print(filenames[i[0]], 'is similar to', filenames[i[1]], '', i[2])

**Question 7) (20 points) Using Spacy’s part of speech tagger, process all sentences (hint: don’t forget to split the reviews) and count how many NOUN and VERB tags are found in all the movies review (TRAINING folder) separating them by label. In other words, how many NOUN and VERB tags are found in positive reviews, and how many NOUN and VERB tags are found in negative reviews. Answer the following question: When comparing both, do you see any differences? Why do you think about the differences? Or lack of them**

In [27]:
nlp = spacy.load("en_core_web_sm")
# total number of nouns and verbs in positive reviews
positive_reviews_files_path = "/content/exam1_dataset/TRAINING/positive/"
# v_count = []
# n_count = []
p_dict = {}
v_count = 0
n_count = 0
# p_count = 0
for i in os.listdir(positive_reviews_files_path):
  file_content = open(os.path.join(positive_reviews_files_path,i), 'r').read()
  doc = nlp(file_content)
  for i in doc:
    if i.pos_=='VERB':
      v_count+=1
    elif i.pos_=='NOUN':
      n_count+=1
    elif i.pos_ == 'PUNCT':
      if i in p_dict:
        p_dict[i.text]+=1
      else:
        p_dict[i.text]=1

In [28]:
print('Total number of NOUNs present in positive reviews are {}'.format(v_count))
print('Total number of VERBS present in positive reviews are {}'.format(n_count))

Total number of NOUNs present in positive reviews are 342125
Total number of VERBS present in positive reviews are 542978


In [29]:
# nlp = spacy.load("en_core_web_sm")
# total number of nouns and verbs in negative reviews
negative_reviews_files = "/content/exam1_dataset/TRAINING/negative/"
# v_count1 = []
# n_count1 = []
p_count1 = {}
v_count1 = 0
n_count1 = 0
count1 = 0
for i in os.listdir(negative_reviews_files):
  file_content = open(os.path.join(negative_reviews_files,i), 'r').read()
  doc = nlp(file_content)
  for i in doc:
    if i.pos_=='VERB':
      v_count1+=1
    elif i.pos_=='NOUN':
      n_count1+=1
    elif i.pos_ == 'PUNCT':
      if i in p_count1:
        p_count1[i.text]+=1
      else:
        p_count1[i.text]=1

In [30]:
print('Total number of NOUNs present in positive reviews are {}'.format(v_count1))
print('Total number of VERBS present in positive reviews are {}'.format(n_count1))

Total number of NOUNs present in positive reviews are 355559
Total number of VERBS present in positive reviews are 528475


- When comparing both, do you see any differences?
  - When I compare the count of NOUNs and VERBs  between POSITIVE and NEGATIVE folder - Yes there is a difference. When comparing them indivisually within each folder / dataset also there is a difference.
- Why do you think about the differences? Or lack of them
  - The difference is primarility because of the nature of the review. 

**Question 8) (20 points) Using the results from the PoS process in question 7, count how many different PUNCT tags are found and their respective counts from the full dataset provided (both negative and positives together). Using regex, write a set of regular expressions that generate the same counts from the dataset without using NLTK or Spacy, just regex. Can you get the same counts? If not, why do you think this is?**

In [45]:
total_punct = len(p_count1) + len(p_dict)
print('Total number of different PUNCT tags found is {}\n'.format(total_punct))
df = pd.DataFrame(p_dict.items(), columns=['PUNCT', 'Count'])
df1 = pd.DataFrame(p_count1.items(), columns=['PUNCT', 'Count'])
df_f = pd.concat([df, df1], ignore_index=True, sort=False)
df8 = df_f.groupby('PUNCT')['Count'].value_counts().to_frame()
df8.columns = ['Counts']
df8 = df8.reset_index().drop('Count', 1).drop_duplicates()
print('The respective counts from the full dataset for each punct is provided in the below dataframe\n')
df8

Total number of different PUNCT tags found is 8462

The respective counts from the full dataset for each punct is provided in the below dataframe



,PUNCT,Counts
0,!,2
1,!!!<br,1
2,!!!and,1
3,!.<br,1
4,!<br,2
...,...,...
6967,’s,2
6968,“,2
6969,”,2
6970,…,1


In [16]:
# Solving PUNT using RE
# merging the file contents to one string
file_content_corpus_merge = [' '.join(file_content_corpus[:])]
# the login used here for RE is that everything apart from word character and space is a PUNCT
punct_re = r'[^\w\s]'
reCount = re.findall(punct_re,file_content_corpus_merge[0])
print('{}'.format(len(reCount)))

1329527

- Can you get the same counts? If not, why do you think this is?
  - No, it is not the same count when done with regular expression and Spacy. Because, Spacy's coverage is both rule-based and contextual.

**Bonus Question: (40 points) Using the code from Class 09 - Word Embeddings, pre-tune BERT in order to classify movie reviews. You can use the full TRAINING folder for the tuning and use the UNLABELED folder for your final classification/prediction task. Do the label predictions from BERT match what your classifier from Question 4 predicted? If they don’t, are they better? Please say why. NOTE: This is a pro-level task and while you might be tempted to copy code from the Internet, do not do it, as you will fail the class if you are plagiarizing. The code from the class can be tweaked to do the task with some adjustments. This is definitely not easy, but that is why it is worth an extra 40 points!**

In [49]:
!pip install transformers

In [ ]:
import tensorflow as tf
import torch
from transformers import BertTokenizer
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
